<a href="https://colab.research.google.com/github/Dkepffl/2022-2-ESAA/blob/main/Assignment/Assignment221107_text5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **CHAPTER 08 텍스트 분석**
___


## **07 문서 군집화 소개와 실습(Opinion Review 데이터 세트)**
___


### | **문서 군집화 개념**
- **문서 군집화(Document Clustering)**는 **비슷한 텍스트 구성의 문서를 군집화(Clustering)하는 것**이다.
- 문서 군집화는 동일한 군집에 속하는 문서를 같은 카테고리 소속으로 분류할 수 있으므로, 앞에서 소개한 텍스트 분류 기반의 문서 분류와유사하다. 하지만 문서 군집화는 학습 데이터셋이 필요없는 비지도학습 기반이라는 점에서 차이가 있다.


### | **Opinion Review 데이터 세트를 이용한 문서 군집화 수행하기**
- 문서 군집화를 수행할 데이터셋은 UCI 머신러닝 리포지토리에 있는 Opinion Review 데이터셋이다.
  + 해당 데이터셋은 51개의 텍스트 파일로 구성되어 있다.
  + 각 파일은 Tripadvisor(호텔), Edmunds.com(자동차), Amazon.com(전자제품) 사이트에서 가져온 리뷰 문서이다.
  + 각 문서는 약 100개 정도의 문장을 가지고 있다.
- 먼저 해당 디렉터리 내의 모든 파일에 대해 각각 for 반복문으로 반복하면서 개별 파일명을 파일명 리스트에 추가하고, 개별 파일은 DataFrame으로 읽은 후, 다시 문자열로 반환하고 파일 내용 리스트에 추가한다.
- 이렇게 만들어진 파일명 리스트와 파일 내용 리스트를 이용해, 새롭게 파일명과 파일 내용을 컬럼으로 가지는 DataFrame을 생성한다.

In [2]:
# import packages
import pandas as pd
import glob ,os

In [6]:
# 디렉터리 경로 설정
path = r'/content/drive/MyDrive/2022-2 ESAA/Data/topics' 

# path로 지정한 디렉토리 밑에 있는 모든 .data 파일들의 파일명을 리스트로 취합
all_files = glob.glob(os.path.join(path, "*.data"))    
filename_list = []
opinion_text = []

In [7]:
# 개별 파일들의 파일명은 filename_list 리스트로
# 개별 파일들의 파일내용은 DataFrame로딩 후 다시 string으로 변환하여 opinion_text 리스트로 
for file_ in all_files:
    # 개별 파일을 읽어서 DataFrame으로 생성 
    df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')
    
    # 절대경로로 주어진 파일명을 가공
    filename_ = file_.split('\\')[-1]
    filename = filename_.split('.')[0]

    # 파일명 리스트와 파일내용 리스트에 파일명과 파일 내용을 추가. 
    filename_list.append(filename.strip('/content/drive/MyDrive/2022-2 ESAA')) # 파일 이름이 너무 길어서 일부 삭제
    opinion_text.append(df.to_string())

In [8]:
# 파일명 리스트와 파일내용 리스트를  DataFrame으로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,ata/topics/accuracy_garmin_nuvi_255W_gps,...
1,ata/topics/bathroom_bestwestern_hotel_sf,...
2,ata/topics/eyesight-issues_amazon_kindl,...
3,ata/topics/comfort_toyota_camry_2007,...
4,ata/topics/display_garmin_nuvi_255W_gps,...


- 각 파일 이름(`filename`) 자체만으로 의견(opinion)의 텍스트(text)가 어떠한 제품/서비스에 대한 리뷰인지 알 수 있다.
- 이제 문서를 TF-IDF 형태로 피처 벡터화한다.
  + tokenizer는 이전 예제에서 Lemmatization을 구현한 `LemNormalize()` 함수를 이용한다.
  + `TfidVectorizer`의 `fit_transform()`의 인자로 데이터프레임 `document_df`의 `opinion_text` 컬럼을 입력하면 개별 문서 텍스트에 대해 TF-IDF 변환된 행렬을 구할 수 있다.

In [9]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

# tokenizer
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [16]:
# TF-IDF 벡터화
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english', ngram_range=(1,2), min_df=0.05, max_df=0.85 )

# opinion_text 컬럼값으로 피처 벡터화 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  % sorted(inconsistent)


- 문서별 텍스트가 TF-IDF 변환된 피처 벡터화 행렬 데이터에서 군집화를 수행하여, 어떤 문서끼리 군집되는지 확인해보자.
- 군집화 알고리즘으로는 **K-평균**을 적용한다.
- 문서의 유형은 크게 전자제품, 자동차, 호텔이 있다. 그리고 전자제품은 다시 네비게이션, 아이팟, 킨들, 랩탑 컴퓨터 등으로 다시 나눌 수 있다.
- 우선 5개의 중심(Centroid) 기반으로 어떻게 군잡화되는지 확인해보자. 최대 반복 횟수 `max_iter`는 1000으로 설정하며, 군집화 후 군집의 Label 값과 중심별로 할당된 데이터셋의 좌표 값을 구한다.

In [17]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)

# 군집의 Label 값과 중심별로 할당된 데이터셋의 좌표값
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

- 각 데이터별로 할당된 군집의 레이블을 파일명과 파일 내용을 가지고 있는 데이터프레임 `document_df`에 `'cluster_label'` 컬럼을 추가해 저장한다.
- 각 파일명은 의견 리뷰에 대한 주제를 나타낸다. 군집이 각 주제별로 유사하게 구성되었는지 알아보자.

In [18]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,ata/topics/accuracy_garmin_nuvi_255W_gps,...,3
1,ata/topics/bathroom_bestwestern_hotel_sf,...,0
2,ata/topics/eyesight-issues_amazon_kindl,...,3
3,ata/topics/comfort_toyota_camry_2007,...,1
4,ata/topics/display_garmin_nuvi_255W_gps,...,3


- 판다스의 `sort_values(by=정렬컬럼명)`를 이용하면, 입력된 컬럼명을 기준으로 데이터를 정렬할 수 있다.
- 이를 이용하여, 각 Label 값별로 어떤 파일명이 매칭되었는지 확인해보자.

In [19]:
document_df[document_df['cluster_label']==0].sort_values(by='filename') # Cluster 0 : 호텔에 대한 리뷰

,filename,opinion_text,cluster_label
1,ata/topics/bathroom_bestwestern_hotel_sf,...,0
34,ata/topics/room_holiday_inn_l,...,0
33,ata/topics/rooms_bestwestern_hotel_sf,...,0
32,ata/topics/rooms_swissotel_chicag,...,0


In [20]:
document_df[document_df['cluster_label']==1].sort_values(by='filename') # Cluster 1 : 자동차에 대한 리뷰

,filename,opinion_text,cluster_label
6,ata/topics/comfort_honda_accord_2008,...,1
3,ata/topics/comfort_toyota_camry_2007,...,1
14,ata/topics/interior_honda_accord_2008,...,1
17,ata/topics/interior_toyota_camry_2007,...,1
29,ata/topics/quality_toyota_camry_2007,...,1
36,ata/topics/seats_honda_accord_2008,...,1


In [21]:
document_df[document_df['cluster_label']==2].sort_values(by='filename') # Cluster 2 : 주로 호텔에 대한 리뷰

,filename,opinion_text,cluster_label
8,ata/topics/food_holiday_inn_l,...,2
21,ata/topics/food_swissotel_chicag,...,2
22,ata/topics/free_bestwestern_hotel_sf,...,2
24,ata/topics/location_bestwestern_hotel_sf,...,2
23,ata/topics/location_holiday_inn_l,...,2
16,ata/topics/parking_bestwestern_hotel_sf,...,2
31,ata/topics/price_holiday_inn_l,...,2
28,ata/topics/service_bestwestern_hotel_sf,...,2
43,ata/topics/service_holiday_inn_l,...,2
46,ata/topics/service_swissotel_hotel_chicag,...,2


In [22]:
document_df[document_df['cluster_label']==3].sort_values(by='filename') # Cluster 3 : 전자기기(주로 차량용 네비게이션)에 대한 리뷰

,filename,opinion_text,cluster_label
0,ata/topics/accuracy_garmin_nuvi_255W_gps,...,3
12,ata/topics/battery-life_amazon_kindl,...,3
10,ata/topics/battery-life_ipod_nano_8gb,...,3
7,ata/topics/battery-life_netbook_1005ha,...,3
11,ata/topics/buttons_amazon_kindl,...,3
13,ata/topics/directions_garmin_nuvi_255W_gps,...,3
4,ata/topics/display_garmin_nuvi_255W_gps,...,3
2,ata/topics/eyesight-issues_amazon_kindl,...,3
9,ata/topics/features_windows7,...,3
5,ata/topics/fonts_amazon_kindl,...,3


In [23]:
document_df[document_df['cluster_label']==4].sort_values(by='filename') # Cluster 4 : 자동차에 대한 리뷰

,filename,opinion_text,cluster_label
20,ata/topics/gas_mileage_toyota_camry_2007,...,4
25,ata/topics/mileage_honda_accord_2008,...,4
18,ata/topics/performance_honda_accord_2008,...,4
41,ata/topics/transmission_toyota_camry_2007,...,4


- 이번에는 3개의 군집으로 군집화해보자.

In [25]:
from sklearn.cluster import KMeans

# 3개의 집합으로 군집화 
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_

document_df['cluster_label'] = cluster_label

In [26]:
document_df[document_df['cluster_label']==0].sort_values(by='cluster_label') # Cluster 0  : 주로 자동차에 관한 리뷰

,filename,opinion_text,cluster_label
3,ata/topics/comfort_toyota_camry_2007,...,0
6,ata/topics/comfort_honda_accord_2008,...,0
14,ata/topics/interior_honda_accord_2008,...,0
17,ata/topics/interior_toyota_camry_2007,...,0
18,ata/topics/performance_honda_accord_2008,...,0
20,ata/topics/gas_mileage_toyota_camry_2007,...,0
25,ata/topics/mileage_honda_accord_2008,...,0
29,ata/topics/quality_toyota_camry_2007,...,0
36,ata/topics/seats_honda_accord_2008,...,0
41,ata/topics/transmission_toyota_camry_2007,...,0


In [27]:
document_df[document_df['cluster_label']==1].sort_values(by='cluster_label') # Cluster 1 : 주로 호텔에 관한 리뷰

,filename,opinion_text,cluster_label
1,ata/topics/bathroom_bestwestern_hotel_sf,...,1
8,ata/topics/food_holiday_inn_l,...,1
16,ata/topics/parking_bestwestern_hotel_sf,...,1
21,ata/topics/food_swissotel_chicag,...,1
22,ata/topics/free_bestwestern_hotel_sf,...,1
23,ata/topics/location_holiday_inn_l,...,1
24,ata/topics/location_bestwestern_hotel_sf,...,1
28,ata/topics/service_bestwestern_hotel_sf,...,1
31,ata/topics/price_holiday_inn_l,...,1
32,ata/topics/rooms_swissotel_chicag,...,1


In [28]:
document_df[document_df['cluster_label']==2].sort_values(by='cluster_label') # Cluster 2 : 주로 전자기기에 관한 리뷰

,filename,opinion_text,cluster_label
0,ata/topics/accuracy_garmin_nuvi_255W_gps,...,2
47,ata/topics/speed_garmin_nuvi_255W_gps,...,2
44,ata/topics/updates_garmin_nuvi_255W_gps,...,2
42,ata/topics/speed_windows7,...,2
40,ata/topics/video_ipod_nano_8gb,...,2
39,ata/topics/size_asus_netbook_1005ha,...,2
38,ata/topics/screen_ipod_nano_8gb,...,2
37,ata/topics/screen_netbook_1005ha,...,2
35,ata/topics/satellite_garmin_nuvi_255W_gps,...,2
30,ata/topics/screen_garmin_nuvi_255W_gps,...,2


### | **군집별 핵심 단어 추출하기**
- 각 군집(Cluster)에 속한 문서는 핵심 단어를 주축으로 군집화되어 있을 것이다. 이번에는 각 군집을 구성하는 핵심 단어가 어떤 것인지 확인해보자.
- `KMeans` 객체는 각 군집을 구성하는 단어 피처가 군집의 중심(Centroid)을 기준으로 얼마나 가깝게 위치해 있는지 `cluster_centers_`라는 속성으로 제공한다.
  + `clusters_centers_` : 배열 값으로 제공된다. 행은 개별 군집을, 열은 개별 피처를 의미한다.
  + 각 배열 내의 값은 개별 군집 내의 상대 위치를 숫자로 표현한 일종의 좌표 값이다.
- 앞에서 군집 3개로 생성한 `KMeans` 객체인 `km_cluster`에서 `cluster_centers` 속성값을 가져와 확인해보자.

In [29]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape :',cluster_centers.shape)
print(cluster_centers)

cluster_centers shape : (3, 4611)
[[0.         0.00092551 0.         ... 0.         0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]
 [0.01005322 0.         0.         ... 0.00706287 0.         0.        ]]


- `cluster_centers_`는 (3, 2409) 형태의 배열이다. 즉, 3개의 군집과 2409개의 word 피처로 구성되어 있다.
- 각 행의 배열 값은 각 군집 내의 피처와 개별 중심 사이의 거리를 상대값으로 나타낸 것이다. 0에서 1까지의 값을 가질 수 있으며, 1에 가까울수록 중심과 가까운 값이다.
- 이제 `cluster_centers_` 속성값을 이용하여, 각 군집별 핵심 단어를 찾아보자.
  + ndarray의 `argsort()[:,::-1]`를 이용하면 `cluster_centers` 배열의 값이 큰 순으로 정렬된 위치 인덱스 값을 반환한다.
  + 새로운 함수 `get_cluster_details()`를 생성하여 위의 과정을 구현해보자. `cluster_centers` 배열 내에서 가장 값이 큰 데이터의 위치 인덱스를 추출하고 해당 인덱스를 이용하여 핵심 단어 이름과 그때의 상대 위치 값을 춫출하여 `cluster_details`라는 Dict 객체 변수에 기록하고 반환한다.

In [30]:
# 군집별 top n 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명들을 반환
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}
    
    # cluster_centers array 의 값이 큰 순으로 정렬된 index 값을 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    #개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화. 
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        # cluster_centers_.argsort()[:,::-1] 로 구한 index 를 이용하여 top n 피처 단어를 구함
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]
        
        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함 
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        
        # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값, 그리고 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames
        
    return cluster_details

- `get_cluster_details()`를 호출하면 dictionary를 원소로 가지는 리스트인 `cluster_details`를 반환한다. 이 리스트에는 개별 군집번호, 핵심 단어, 핵심 단어 중심 위치 상대값, 파일명 속성값 정보가 있다.
- 이 정보들을 보기 좋게 표현하기 위해 별도의 `print_cluster_details()` 함수를 만들어보자.

In [31]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('####### Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print('Reviews 파일명 :',cluster_detail['filenames'][:7])
        print('==================================================')

- 이제 위에서 생성한 함수들을 호출해보자.

In [32]:
feature_names = tfidf_vect.get_feature_names()
cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,feature_names=feature_names, clusters_num=3, top_n_features=10 )
print_cluster_details(cluster_details)

####### Cluster 0
Top features: ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reviews 파일명 : ['ata/topics/comfort_toyota_camry_2007', 'ata/topics/comfort_honda_accord_2008', 'ata/topics/interior_honda_accord_2008', 'ata/topics/interior_toyota_camry_2007', 'ata/topics/performance_honda_accord_2008', 'ata/topics/gas_mileage_toyota_camry_2007', 'ata/topics/mileage_honda_accord_2008']
####### Cluster 1
Top features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews 파일명 : ['ata/topics/bathroom_bestwestern_hotel_sf', 'ata/topics/food_holiday_inn_l', 'ata/topics/parking_bestwestern_hotel_sf', 'ata/topics/food_swissotel_chicag', 'ata/topics/free_bestwestern_hotel_sf', 'ata/topics/location_holiday_inn_l', 'ata/topics/location_bestwestern_hotel_sf']
####### Cluster 2
Top features: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


- Cluster #0에서는 'interior', 'seat' 등이 핵심 단어로 군집화되었다.
- Cluster #1에서는 'room', 'hotel', 'service' 등 방과 서비스 등이 핵심 단어로 군집화되었다.
- Cluster #2에서는 'screen', 'battery', 'keyboard' 등과 같이 화면, 배터리 수명 등이 핵심단어로 군집화되었다.

## **10 텍스트 분석 실습-캐글 Mercari Price Suggestion Challenge**
___
- Mercari Price Suggestion Challenge는 캐글에서 진행된, 일본의 대형 온라인 쇼핑몰인 Mercari 사의 제품에 대해 가격을 예측하는 과제이다.
- 제공되는 데이터셋은 제품에 대한 여러 속성 및 제품 설명 등의 텍스트로 구성되어 있다.
- 제공되는 데이터셋의 피처는 다음과 같다.
  - `train_id` : 데이터 id
  - `name` : 제품명
  - `item_condition_id` : 판매자가 제공하는 제품 상태
  - `category_name` : 카테고리 명
  - `brand_name` : 브랜드 이름
  - `price` : 제품 가격. 타겟값
  - `shipping` : 배송비 무료 여부. 1이면 무료, 0이면 유료
  - `item_description` : 제품에 대한 설명
- 이 예제는 텍스트 형태의 비정형 데이터와 다른 정형 데이터를 같이 활용하여 회귀를 수행한다.


### | **데이터 전처리**
- 먼저 데이터셋을 로딩하고 데이터를 간략하게 살펴보자.

In [ ]:
from sklearn.linear_model import Ridge , LogisticRegression
from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
import pandas as pd

mercari_df= pd.read_csv('mercari_train.tsv',sep='\t')
print(mercari_df.shape)
mercari_df.head(3)

- 피처 타입과 Null 여부를 확인해보자.

In [ ]:
print(mercari_df.info())

- `brand_name` 컬럼의 경우, 가격에 영향을 미치는 중요 요인으로 판단되지만, 매우 많은 Null 값을 가지고 있다. 
- `category_name`은 약 63000건의 null 데이터를 가지고 있다.
- `item_description`의 null 값은 4건으로 미비하다.
- Target 값인 `price` 컬럼의 데이터 분포를 살펴보자.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

y_train_df = mercari_df['price']
plt.figure(figsize=(6,4))
sns.distplot(y_train_df,kde=False)

- `price` 컬럼의 값은 비교적 적은 가격을 가진 데이터 값이 많아 왜곡되어 분포하고 있다. 로그 변환을 적용한 후, 다시 분포를 살펴보자.

In [ ]:
import numpy as np

y_train_df = np.log1p(y_train_df)
sns.distplot(y_train_df,kde=False)

- 로그 값으로 변환하면 price 값이 비교적 정규 분포에 가까운 데이터를 가지게 된다. 데이터셋의 price 컬럼을 로그 변환한 값으로 변경하자.

In [ ]:
mercari_df['price'] = np.log1p(mercari_df['price'])
mercari_df['price'].head(3)

- 다른 피처의 값도 살펴보면, `shipping`과 `item_condition_id` 값의 분포는 다음과 같다.

In [ ]:
print('Shipping 값 유형:\n',mercari_df['shipping'].value_counts())

- `Shipping` 컬럼은 배송비 유무를 의미하는 컬럼으로 값이 비교적 균일하다.

In [ ]:
print('item_condition_id 값 유형:\n',mercari_df['item_condition_id'].value_counts())

- `item_condition_id`는 판매자가 제공하는 제품 상태로, 캐글에 설명이 없어 정확히 그 값이 의미하는 의미는 알 수 없으나, 1, 2, 3이 주를 이르고 있다.
- `item_condition_id` 컬럼은 Null 값은 별로 없지만, 제품에 대해 별도의 설명이 없는 경우 'No description yet'으로 되어 있다. 이 값이 얼마나 있는지 확인해보자.

In [ ]:
boolean_cond= mercari_df['item_description']=='No description yet'
mercari_df[boolean_cond]['item_description'].count()

-  'No description yet' 값을 가지고 있는 행은 총 82489개이다. 이 값은 Null과 마찬가지로 의미 있는 피처값으로 사용할 수 없어, 적절한 값으로 변경해야 한다.
- 이번에는 `category_name`을 살펴보자.
  + `category_name` 컬럼의 값을 '/'을 기준으로 대, 중 소분류 단위로 분리한다. 이를 수행하는 함수를 `split_cat()`이라고 하자.
  + 데이터프레임의 apply lambda 식을 적용한다.

In [ ]:
# apply lambda에서 호출되는 대,중,소 분할 함수 생성, 대,중,소 값을 리스트 반환
def split_cat(category_name):
    try:
        return category_name.split('/')
    except:
        return ['Other_Null' , 'Other_Null' , 'Other_Null']

# 위의 split_cat( )을 apply lambda에서 호출하여 대,중,소 컬럼을 mercari_df에 생성. 
mercari_df['cat_dae'], mercari_df['cat_jung'], mercari_df['cat_so'] = \
                        zip(*mercari_df['category_name'].apply(lambda x : split_cat(x)))

# 대분류만 값의 유형과 건수를 살펴보고, 중분류, 소분류는 값의 유형이 많으므로 분류 갯수만 추출
print('대분류 유형 :\n', mercari_df['cat_dae'].value_counts())
print('중분류 갯수 :', mercari_df['cat_jung'].nunique())
print('소분류 갯수 :', mercari_df['cat_so'].nunique())

- 대분류의 경우, Women, Beauty, Kids 등 분류가 매우 많다. 중분류는 114개, 소분류는 817개로 되어 있다.
- 마지막으로 `brand_naem`, `category_name`, `item_description` 컬럼의 Null 값을 일괄적으로 'Other Null`로 동일하게 변경한다.

In [ ]:
mercari_df['brand_name'] = mercari_df['brand_name'].fillna(value='Other_Null')
mercari_df['category_name'] = mercari_df['category_name'].fillna(value='Other_Null')
mercari_df['item_description'] = mercari_df['item_description'].fillna(value='Other_Null')

In [ ]:
# 각 컬럼별로 Null값 건수 확인
mercari_df.isnull().sum()

### | **피처 인코딩과 피처 벡터화**
- Mercari Price Suggestion에 이용되는 데이터셋은 문자열 컬럼이 많다. 이 문자열 컬럼 중 레이블 또는 원-핫 인코딩을 수행하거나 피처 벡터화로 변환할 컬럼을 선별해보자.
- 선형 회귀 모델과 회귀 트리 모델을 이용할 것이기 때문에, 선형 회귀에서 선호되는 원-핫 인코딩을 적용한다.
- 피처 벡터화의 경우, 짧은 텍스트의 경우 Count 기반 벡터화, 장문의 텍스트는 TD-IDF 벡터화를 적용한다.


#### **brand_name**
- 먼저 상품의 브랜드명을 가지고 있는 `brand_name` 컬럼의 값 유형과 대표적인 값을 5개 정도 살펴보자. 

In [ ]:
print('brand name 의 유형 건수 :', mercari_df['brand_name'].nunique())
print('brand name sample 5건 : \n', mercari_df['brand_name'].value_counts()[:5])

- `brand_name` 컬럼의 경우, 대부분 명료한 문자열로 되어 있어 별도의 피처 벡터화 없이 원-핫 인코딩을 적용하면 된다.

#### **name**
- 상품의 이름을 의미하는 `name` 컬럼을 살펴보자.

In [ ]:
print('name 의 종류 갯수 :', mercari_df['name'].nunique())
print('name sample 7건 : \n', mercari_df['name'][:7])

- `name` 컬럼은 값의 종류가 매우 다양하다.그리고 단어 위주의 짧은 텍스트 형태로 되어있다. 따라서 Count 기반 벡터화를 적용한다.

#### **category_name**
- `category_name` 컬럼은 이전에 전처리를 통해, 대, 중, 소분류 값을 가지고 있는 세 개의 컬럼으로 분리되었다.
- 이 컬럼에 각각 원-핫 인코딩을 적용한다.

#### **shipping & item_condition_id**
- `shipping` : 배송비 무료 여부를 나타내는 컬럼으로 0과 1 두가지 유형의 값을 가지고 있다.
- `item_condition_id` : 상품 상태에 관한 컬럼으로 1, 2, 3, 4, 5 다섯 가지 유형의 값을 가지고 있다.
- 두 컬럼 모두 원-핫 인코딩을 적용한다.


#### **item_description**
- 상품에 대한 간단 설명을 포함하는 이 컬럼은 데이터셋에서 가장 긴 텍스트를 가지고 있다.
- 해당 컬럼의 평균 문자열 크기와 컬럼에 속한 텍스트를 2개 정도 추출해보자.

In [ ]:
pd.set_option('max_colwidth', 200)

# item_description의 평균 문자열 크기
print('item_description 평균 문자열 크기:',mercari_df['item_description'].str.len().mean())

mercari_df['item_description'][:2] # 텍스트 2개 정도 추출

- 평균 문자열이 145자로 비교적 크므로, 해당 컬럼은 TF-IDF 피처 벡터화한다.

#### **주요 컬럼 인코딩 및 피처 벡터화**
- `name`과 `item_description` 컬럼 피처 벡터화

In [ ]:
# name 컬럼 피처 벡터화
cnt_vec = CountVectorizer()
X_name = cnt_vec.fit_transform(mercari_df.name)

# item_description 컬럼 피처 벡터화 
tfidf_descp = TfidfVectorizer(max_features = 50000, ngram_range= (1,3) , stop_words='english')
X_descp = tfidf_descp.fit_transform(mercari_df['item_description'])

print('name vectorization shape:',X_name.shape)
print('item_description vectorization shape:',X_descp.shape)

- `LabelBinarizer`로 희소 행렬 형태로 원-핫 인코딩

In [ ]:
from sklearn.preprocessing import LabelBinarizer

# brand_name, item_condition_id, shipping 각 피처들을 희소 행렬 원-핫 인코딩 변환
lb_brand_name= LabelBinarizer(sparse_output=True)
X_brand = lb_brand_name.fit_transform(mercari_df['brand_name'])

lb_item_cond_id = LabelBinarizer(sparse_output=True)
X_item_cond_id = lb_item_cond_id.fit_transform(mercari_df['item_condition_id'])

lb_shipping= LabelBinarizer(sparse_output=True)
X_shipping = lb_shipping.fit_transform(mercari_df['shipping'])

# cat_dae, cat_jung, cat_so 각 피처들을 희소 행렬 원-핫 인코딩 변환
lb_cat_dae = LabelBinarizer(sparse_output=True)
X_cat_dae= lb_cat_dae.fit_transform(mercari_df['cat_dae'])

lb_cat_jung = LabelBinarizer(sparse_output=True)
X_cat_jung = lb_cat_jung.fit_transform(mercari_df['cat_jung'])

lb_cat_so = LabelBinarizer(sparse_output=True)
X_cat_so = lb_cat_so.fit_transform(mercari_df['cat_so'])

- 제대로 변환됐는지 인코딩 데이터셋과 데이터 타입, shape를 확인해보자.

In [ ]:
print(type(X_brand), type(X_item_cond_id), type(X_shipping))
print('X_brand_shape:{0}, X_item_cond_id shape:{1}'.format(X_brand.shape, X_item_cond_id.shape))
print('X_shipping shape:{0}, X_cat_dae shape:{1}'.format(X_shipping.shape, X_cat_dae.shape))
print('X_cat_jung shape:{0}, X_cat_so shape:{1}'.format(X_cat_jung.shape, X_cat_so.shape))

- 인코딩된 데이터셋이 CSR 형태로 변환된 `csr_matrix` 타입이다.
- 이제 앞에서 피처 벡터화한 데이터셋과 희소행렬 형태로 원-핫 인코딩한 데이터셋을 `hstack()`을 이용하여 결합해보자.

In [ ]:
from  scipy.sparse import hstack
import gc

sparse_matrix_list = (X_name, X_descp, X_brand, X_item_cond_id,
            X_shipping, X_cat_dae, X_cat_jung, X_cat_so)

# 사이파이 sparse 모듈의 hstack 함수를 이용하여 앞에서 인코딩과 Vectorization을 수행한 데이터 셋을 모두 결합
X_features_sparse= hstack(sparse_matrix_list).tocsr()
print(type(X_features_sparse), X_features_sparse.shape)

# 데이터 셋이 메모리를 많이 차지하므로 사용 용도가 끝났으면 바로 메모리에서 삭제
del X_features_sparse
gc.collect()

### | **릿지 회귀 모델 구축 및 평가**
- 먼저 모델을 평가하는 함수를 만들어보자. RMSLE를 평가 지표로 활용한다.
$$RMSLE = \sqrt{\frac{1}{n}\sum^n_{i=1}(log(p_i + 1) - log(a_i +1))^2}$$
- 학습 데이터셋에서 `price` 컬럼을 로그변환했기 때문에, 예측된 price 값을 지수 변환해주어야 한다.

In [ ]:
def rmsle(y , y_pred):
    # underflow, overflow를 막기 위해 log가 아닌 log1p로 rmsle 계산 
    return np.sqrt(np.mean(np.power(np.log1p(y) - np.log1p(y_pred), 2)))

def evaluate_org_price(y_test , preds): 
    
    # 원본 데이터는 log1p로 변환되었으므로 exmpm1으로 원복 필요. 
    preds_exmpm = np.expm1(preds)
    y_test_exmpm = np.expm1(y_test)
    
    # rmsle로 RMSLE 값 추출
    rmsle_result = rmsle(y_test_exmpm, preds_exmpm)
    return rmsle_result

- 이제 학습용 데이터셋을 생성하고, 모델을 학습/예측하는 함수를 만들어보자. 
- 이 함수는 `model` 인자로 사이킷런의 회귀 estimator 객체를, `matirx_list`인자로 최종 데이터셋을 결합한 희소 행렬 리스트를 입력받는다.
- 평가 데이터셋은 전체 데이터의 20%로 한다.

In [ ]:
import gc 
from  scipy.sparse import hstack

def model_train_predict(model,matrix_list):
    # scipy.sparse 모듈의 hstack 을 이용하여 sparse matrix 결합
    X= hstack(matrix_list).tocsr()     
    
    X_train, X_test, y_train, y_test=train_test_split(X, mercari_df['price'], 
                                                      test_size=0.2, random_state=156)
    
    # 모델 학습 및 예측
    model.fit(X_train , y_train)
    preds = model.predict(X_test)
    
    del X , X_train , X_test , y_train 
    gc.collect()
    
    return preds , y_test

- 앞에서 만든 함수를 이용하여, Ridge 모델로 Mercari Price의 회귀 예측을 수행해보자.

In [ ]:
linear_model = Ridge(solver = "lsqr", fit_intercept=False)

sparse_matrix_list = (X_name, X_brand, X_item_cond_id,
                      X_shipping, X_cat_dae, X_cat_jung, X_cat_so)
linear_preds , y_test = model_train_predict(model=linear_model ,matrix_list=sparse_matrix_list)
print('Item Description을 제외했을 때 rmsle 값:', evaluate_org_price(y_test , linear_preds))

sparse_matrix_list = (X_descp, X_name, X_brand, X_item_cond_id,
                      X_shipping, X_cat_dae, X_cat_jung, X_cat_so)
linear_preds , y_test = model_train_predict(model=linear_model , matrix_list=sparse_matrix_list)
print('Item Description을 포함한 rmsle 값:',  evaluate_org_price(y_test ,linear_preds))

### | **LightGBM 회귀 모델 구축과 앙상블을 이용한 최종 예측 평가**
- 이번에는 LightGBM을 이용하여 회귀를 수행한 뒤, 릿지 모델의 예측값과 LightGBM 모델의 예측값을 간단한 앙상블 방식으로 섞어 최종 회귀 예측값을 평가해보자.

In [ ]:
from lightgbm import LGBMRegressor

sparse_matrix_list = (X_descp, X_name, X_brand, X_item_cond_id,
                      X_shipping, X_cat_dae, X_cat_jung, X_cat_so)

lgbm_model = LGBMRegressor(n_estimators=200, learning_rate=0.5, num_leaves=125, random_state=156)
lgbm_preds , y_test = model_train_predict(model = lgbm_model , matrix_list=sparse_matrix_list)
print('LightGBM rmsle 값:',  evaluate_org_price(y_test , lgbm_preds))

In [ ]:
preds = lgbm_preds * 0.45 + linear_preds * 0.55
print('LightGBM과 Ridge를 ensemble한 최종 rmsle 값:',  evaluate_org_price(y_test , preds))